## Setup

**`Runtime` -->  `Change runtime type` --> `GPU`**

In [ ]:
! curl https://getcroc.schollz.com | bash

! git clone https://github.com/bird-feeder/BirdFSD-YOLOv5.git
%cd BirdFSD-YOLOv5

! pip -q install -r requirements.txt
! pip -q install "PyYAML>=5.3.1" wandb

! git clone https://github.com/ultralytics/yolov5.git

In [ ]:
#@title Login to W&B
! wandb login

**Send the `.env` file to the notebook:**

- Using [`Croc`](https://github.com/schollz/croc):

  - Run the following command from the root folder where `BirdFSD-YOLOv5` is cloned:

    ```shell
    croc send "BirdFSD-YOLOv5/.env"
    ```

  - Copy the passphrase part and paste it in the cell input below.

- If not using `Croc`:
  - Drag and drop the `.env` file inside `BirdFSD-YOLOv5` in this notebook and skip the next cell.

In [ ]:
PASSPHRASE = '' #@param {'type': 'string'}
! croc --yes $PASSPHRASE

## Download model weights

In [ ]:
! python model_utils/download_weights.py --model-version latest --output 'best.pt'

## Data Preprocessing

In [ ]:
LABEL_STUDIO_PROJECT_ID=  #required
! python json2yolov5.py -p $LABEL_STUDIO_PROJECT_ID

! mv dataset-YOLO/dataset_config.yml .
! python utils/relative_to_abs.py

## Available Training Options

```
usage: train.py [-h] [--weights WEIGHTS] [--cfg CFG] [--data DATA] [--hyp HYP]
                [--epochs EPOCHS] [--batch-size BATCH_SIZE] [--imgsz IMGSZ]
                [--rect] [--resume [RESUME]] [--nosave] [--noval]
                [--noautoanchor] [--noplots] [--evolve [EVOLVE]]
                [--bucket BUCKET] [--cache [CACHE]] [--image-weights]
                [--device DEVICE] [--multi-scale] [--single-cls]
                [--optimizer {SGD,Adam,AdamW}] [--sync-bn] [--workers WORKERS]
                [--project PROJECT] [--name NAME] [--exist-ok] [--quad]
                [--cos-lr] [--label-smoothing LABEL_SMOOTHING]
                [--patience PATIENCE] [--freeze FREEZE [FREEZE ...]]
                [--save-period SAVE_PERIOD] [--local_rank LOCAL_RANK]
                [--entity ENTITY] [--upload_dataset [UPLOAD_DATASET]]
                [--bbox_interval BBOX_INTERVAL]
                [--artifact_alias ARTIFACT_ALIAS]

optional arguments:
  -h, --help            show this help message and exit
  --weights WEIGHTS     initial weights path
  --cfg CFG             model.yaml path
  --data DATA           dataset.yaml path
  --hyp HYP             hyperparameters path
  --epochs EPOCHS
  --batch-size BATCH_SIZE
                        total batch size for all GPUs, -1 for autobatch
  --imgsz IMGSZ, --img IMGSZ, --img-size IMGSZ
                        train, val image size (pixels)
  --rect                rectangular training
  --resume [RESUME]     resume most recent training
  --nosave              only save final checkpoint
  --noval               only validate final epoch
  --noautoanchor        disable AutoAnchor
  --noplots             save no plot files
  --evolve [EVOLVE]     evolve hyperparameters for x generations
  --bucket BUCKET       gsutil bucket
  --cache [CACHE]       --cache images in "ram" (default) or "disk"
  --image-weights       use weighted image selection for training
  --device DEVICE       cuda device, i.e. 0 or 0,1,2,3 or cpu
  --multi-scale         vary img-size +/- 50%
  --single-cls          train multi-class data as single-class
  --optimizer {SGD,Adam,AdamW}
                        optimizer
  --sync-bn             use SyncBatchNorm, only available in DDP mode
  --workers WORKERS     max dataloader workers (per RANK in DDP mode)
  --project PROJECT     save to project/name
  --name NAME           save to project/name
  --exist-ok            existing project/name ok, do not increment
  --quad                quad dataloader
  --cos-lr              cosine LR scheduler
  --label-smoothing LABEL_SMOOTHING
                        Label smoothing epsilon
  --patience PATIENCE   EarlyStopping patience (epochs without improvement)
  --freeze FREEZE [FREEZE ...]
                        Freeze layers: backbone=10, first3=0 1 2
  --save-period SAVE_PERIOD
                        Save checkpoint every x epochs (disabled if < 1)
  --local_rank LOCAL_RANK
                        DDP parameter, do not modify
  --entity ENTITY       W&B: Entity
  --upload_dataset [UPLOAD_DATASET]
                        W&B: Upload data, "val" option
  --bbox_interval BBOX_INTERVAL
                        W&B: Set bounding-box image logging interval
  --artifact_alias ARTIFACT_ALIAS
                        W&B: Version of dataset artifact to use
```

## Train

In [ ]:
EPOCHS = 30 #@param {type:"integer"}
BATCH_SIZE = 16 #@param {type:"integer"}
PRETRAINED_WEIGHTS = 'best.pt'
IMAGE_SIZE = 768  # input images height

Notes on resuming training:
- If your training was interrupted for any reason you may continue where you left off using the `--resume` flag.
- If your training fully completed, you can start a new training from any model using the `--weights` flag.

In [ ]:
! python yolov5/train.py --img-size $IMAGE_SIZE --batch $BATCH_SIZE --epochs $EPOCHS \
    --data 'dataset_config.yml' --weights $PRETRAINED_WEIGHTS

## Add Additional Metadata

- Click on the run link in the output above.  Paste the run path in the input field, then run the next cell.

In [ ]:
#@markdown **Run path** `<entity>/<project>/<run_id>`
#@markdown ![run_path](https://d.aibird.me/41a26c3b.png)

RUN_PATH = '' #@param {type: 'string'}

! python model_utils/update_run_cfg.py --run-path "$RUN_PATH" --dataset-dir 'dataset-YOLO'

## Cleaning up

1. Clear all the input fields.
2. Clear the output: `Edit` -> `Clear all output`.